<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=b499d5486398d22d4553dd2d68a3571eb0150d07deeff6c542e9f523c504270b
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66


   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-29 09:43:24
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  4.38 K
Current:  1.36 C
-------------------
Today PnL: 19.59 K (0.14%)
Current PnL: -23.10 L (-15.54%)
CY Booked + Current PnL: -11.66 L (-7.85%)
-------------------
Total profit:  87.43 K
Total loss:  -23.98 L
-------------------
Total Booked + Current PnL: 15.32 L (12.59%)
Total Booked PnL: 38.42 L (31.57%)
Curr Year Booked PnL: 11.44 L (8.38%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.25 C
Est FTT PnL: 88.97 L (65.19%)
Deployed:  1.22 C
Current:  1.36 C
CAGR/XIRR %: 6.92%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,94.02,38.0,M-SC,2.46,83617.0,-17160.0,17250.0,0.53,-17.03,20.63,0.09,245.0,-0.99,0.61,9.85,OX40N,NTT,DURABLES
14,BLUESTARCO,2080.00,8.71,45.0,H-MC,3.60,187610.0,22940.0,20393.0,-0.74,13.93,10.87,26.31,89.0,1.12,1.37,23.03,X40N,NTT,AC
50,NATIONALUM,244.55,-46.15,52.0,H-MC,11.87,108162.0,7658.0,21449.0,1.74,7.62,19.83,28.96,79.0,0.36,0.79,44.18,MH,ATH,METALS
42,ITC,452.00,-38.77,45.0,H-LC,1.92,197947.0,-2191.0,23081.0,-0.07,-1.09,11.66,10.44,4.0,-0.09,1.45,4.41,X40,NTT,FMCG
57,RELIANCE,1533.00,-13.66,48.0,H-LC,3.94,216029.0,5663.0,23115.0,0.52,2.69,10.70,13.68,37.0,0.24,1.58,19.66,XY25,NTT,REFINERIES


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAMMAANCAP,326.00,-177.27,70.0,M-SC,6.60,89400.0,-12810.0,106198.0,8.11,-12.53,118.79,91.37,208.0,-0.12,0.66,45.13,XY25,NTT,FINANCE
32,HINDZINC,730.22,23.40,57.0,M-LC,2.93,200883.0,-4193.0,116753.0,2.83,-2.04,58.12,54.89,52.0,-0.04,1.47,21.56,X5K,ATH,METALS
10,BANDHANBNK,400.00,-14.68,41.0,H-SC,4.13,212765.0,-65794.0,323233.0,2.43,-23.62,151.92,92.42,151.0,-0.20,1.56,22.96,XY24,NTT,BANKS
64,SHALBY,327.00,1182.30,64.0,M-SC,13.98,174317.0,-6826.0,52295.0,1.78,-3.77,30.00,25.10,235.0,-0.13,1.28,39.70,XY24,NTT,HEALTHCARE
50,NATIONALUM,244.55,-46.15,52.0,H-MC,11.87,108162.0,7658.0,21449.0,1.74,7.62,19.83,28.96,79.0,0.36,0.79,44.18,MH,ATH,METALS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
30,HCLTECH,1943.91,-4.09,34.0,H-LC,9.95,219062.0,-22862.0,86135.0,-2.00,-9.45,39.32,26.15,8.0,-0.27,1.61,3.52,X40,ATH,IT
13,BERGEPAINT,680.00,-23.00,34.0,H-MC,4.23,207158.0,-20181.0,68238.0,-1.91,-8.88,32.94,21.14,106.0,-0.30,1.52,16.76,XY24,NTT,PAINTS
31,HINDUNILVR,2922.00,-11.98,33.0,H-LC,8.96,247210.0,-9853.0,44992.0,-1.58,-3.83,18.20,13.67,24.0,-0.22,1.81,15.79,XY25,NTT,FMCG
26,GREENPANEL,537.00,197.80,41.0,M-SC,1.03,132834.0,-50244.0,129221.0,-1.38,-27.44,97.28,43.14,230.0,-0.39,0.97,24.95,XY24,NTT,MISC
15,BSOFT,836.99,-19.05,33.0,H-SC,13.78,90831.0,-49822.0,122604.0,-1.37,-35.42,134.98,51.74,131.0,-0.41,0.67,3.58,XR,ATH,IT


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WHIRLPOOL,2270.0,-45.59,29.0,M-SC,1.39,92018.0,520.0,78234.0,-0.46,0.57,85.02,86.07,223.0,0.01,0.67,33.63,XR,NTT,DURABLES


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,90.20,47.0,H-SC,10.25,120231.0,-22248.0,41768.0,0.79,-15.61,34.74,13.70,163.0,-0.53,0.88,41.91,OX40N,NTT,BANKS
77,TTKPRESTIG,770.0,94.02,38.0,M-SC,2.46,83617.0,-17160.0,17250.0,0.53,-17.03,20.63,0.09,245.0,-0.99,0.61,9.85,OX40N,NTT,DURABLES
45,KANSAINER,340.0,-68.16,49.0,H-SC,1.52,219933.0,-49734.0,86060.0,-0.90,-18.44,39.13,13.47,138.0,-0.58,1.61,11.24,XY24,NTT,PAINTS
18,CERA,9475.0,-22.91,36.0,H-SC,1.70,140691.0,-35212.0,77239.0,-1.01,-20.02,54.90,23.89,149.0,-0.46,1.03,21.51,OX40N,NTT,CERAMICS
66,SIS,528.0,2076.68,39.0,H-SC,4.02,87835.0,-23197.0,46807.0,-0.22,-20.89,53.29,21.26,156.0,-0.50,0.64,18.67,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,228.00,45.15,56.0,M-MC,10.74,228992.0,4030.0,163615.0,1.50,1.79,71.45,74.53,192.0,0.02,1.68,34.62,XY24,BTT,STEEL
14,BLUESTARCO,2080.00,8.71,45.0,H-MC,3.60,187610.0,22940.0,20393.0,-0.74,13.93,10.87,26.31,89.0,1.12,1.37,23.03,X40N,NTT,AC
84,VOLTAS,1530.00,-3.88,40.0,H-MC,2.82,201765.0,10023.0,27743.0,0.37,5.23,13.75,19.69,99.0,0.36,1.48,12.38,XY25,NTT,AC
17,CAMS,5211.76,-6.71,42.0,H-SC,1.81,106095.0,4091.0,39839.0,-0.29,4.01,37.55,43.06,122.0,0.10,0.78,21.81,X40N,ATH,MISC
79,UNIONBANK,163.00,-14.73,53.0,M-LC,7.02,154552.0,13712.0,30771.0,1.38,9.74,19.91,31.59,66.0,0.45,1.13,38.80,XY24,NTT,BANKS


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-46.15,52.0,H-MC,11.87,108162.0,7658.0,21449.0,1.74,7.62,19.83,28.96,79.0,0.36,0.79,44.18,MH,ATH,METALS
11,BANKINDIA,190.00,-29.19,54.0,H-MC,8.42,181684.0,1876.0,110537.0,1.55,1.04,60.84,62.52,88.0,0.02,1.33,35.49,XR,NTT,BANKS
37,INDIAMART,4850.92,-55.01,26.0,H-SC,9.99,125472.0,2136.0,131620.0,0.03,1.73,104.90,108.45,119.0,0.02,0.92,24.47,AR,ATH,MISC
85,WHIRLPOOL,2270.00,-45.59,29.0,M-SC,1.39,92018.0,520.0,78234.0,-0.46,0.57,85.02,86.07,223.0,0.01,0.67,33.63,XR,NTT,DURABLES
86,WIPRO,420.00,-16.06,37.0,M-LC,5.47,147808.0,-3137.0,112585.0,1.12,-2.08,76.17,72.51,53.0,-0.03,1.08,3.82,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,ASIANPAINT,3465.66,-18.03,29.0,H-LC,6.53,206844.0,-44924.0,98127.0,0.33,-17.84,47.44,21.13,27.0,-0.46,1.52,11.26,X40,ATH,PAINTS
44,JIOFIN,387.00,-5.69,31.0,H-LC,12.81,256366.0,-10999.0,76448.0,0.76,-4.11,29.82,24.48,48.0,-0.14,1.88,48.60,XY24,BTT,FINANCE
74,TCS,4476.75,-31.26,31.0,H-LC,12.16,274057.0,-71899.0,146758.0,0.57,-20.78,53.55,21.64,1.0,-0.49,2.01,0.57,X40,ATH,IT
31,HINDUNILVR,2922.00,-11.98,33.0,H-LC,8.96,247210.0,-9853.0,44992.0,-1.58,-3.83,18.20,13.67,24.0,-0.22,1.81,15.79,XY25,NTT,FMCG
83,VBL,671.64,-20.49,33.0,H-LC,8.50,286271.0,-29571.0,142248.0,0.89,-9.36,49.69,35.67,5.0,-0.21,2.10,3.19,X40N,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
48,LTIM,7230.2,-7.51,40.0,H-LC,1.57,229050.0,-17101.0,96316.0,0.30,-6.95,42.05,32.18,16.0,-0.18,1.68,26.59,X200,ATH,IT
42,ITC,452.0,-38.77,45.0,H-LC,1.92,197947.0,-2191.0,23081.0,-0.07,-1.09,11.66,10.44,4.0,-0.09,1.45,4.41,X40,NTT,FMCG
65,SIEMENS,4671.5,-3.04,43.0,H-LC,2.40,155880.0,-30215.0,77691.0,0.69,-16.24,49.84,25.51,15.0,-0.39,1.14,14.87,AR,ATH,ELECTRICAL
57,RELIANCE,1533.0,-13.66,48.0,H-LC,3.94,216029.0,5663.0,23115.0,0.52,2.69,10.70,13.68,37.0,0.24,1.58,19.66,XY25,NTT,REFINERIES
19,CIPLA,1795.0,-22.52,40.0,H-LC,4.29,206747.0,2247.0,39179.0,0.65,1.10,18.95,20.25,10.0,0.06,1.52,9.93,X40N,BTT,PHARMA


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-31.26,31.0,H-LC,12.16,274057.0,-71899.0,146758.0,0.57,-20.78,53.55,21.64,1.0,-0.49,2.01,0.57,X40,ATH,IT
80,UNITDSPR,1644.00,-14.51,43.0,H-LC,7.20,223996.0,-10148.0,58777.0,0.38,-4.33,26.24,20.77,86.0,-0.17,1.64,2.28,X40N,NTT,BREWERIES
83,VBL,671.64,-20.49,33.0,H-LC,8.50,286271.0,-29571.0,142248.0,0.89,-9.36,49.69,35.67,5.0,-0.21,2.10,3.19,X40N,ATH,FMCG
30,HCLTECH,1943.91,-4.09,34.0,H-LC,9.95,219062.0,-22862.0,86135.0,-2.00,-9.45,39.32,26.15,8.0,-0.27,1.61,3.52,X40,ATH,IT
42,ITC,452.00,-38.77,45.0,H-LC,1.92,197947.0,-2191.0,23081.0,-0.07,-1.09,11.66,10.44,4.0,-0.09,1.45,4.41,X40,NTT,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.50,-3.04,43.0,H-LC,2.40,155880.0,-30215.0,77691.0,0.69,-16.24,49.84,25.51,15.0,-0.39,1.14,14.87,AR,ATH,ELECTRICAL
42,ITC,452.00,-38.77,45.0,H-LC,1.92,197947.0,-2191.0,23081.0,-0.07,-1.09,11.66,10.44,4.0,-0.09,1.45,4.41,X40,NTT,FMCG
19,CIPLA,1795.00,-22.52,40.0,H-LC,4.29,206747.0,2247.0,39179.0,0.65,1.10,18.95,20.25,10.0,0.06,1.52,9.93,X40N,BTT,PHARMA
5,ASIANPAINT,3465.66,-18.03,29.0,H-LC,6.53,206844.0,-44924.0,98127.0,0.33,-17.84,47.44,21.13,27.0,-0.46,1.52,11.26,X40,ATH,PAINTS
57,RELIANCE,1533.00,-13.66,48.0,H-LC,3.94,216029.0,5663.0,23115.0,0.52,2.69,10.70,13.68,37.0,0.24,1.58,19.66,XY25,NTT,REFINERIES


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-31.26,31.0,H-LC,12.16,274057.0,-71899.0,146758.0,0.57,-20.78,53.55,21.64,1.0,-0.49,2.01,0.57,X40,ATH,IT
40,INFY,2275.00,-20.32,42.0,H-LC,8.18,310788.0,-2100.0,173793.0,0.70,-0.67,55.92,54.87,3.0,-0.01,2.28,5.93,X40,BTT,IT
42,ITC,452.00,-38.77,45.0,H-LC,1.92,197947.0,-2191.0,23081.0,-0.07,-1.09,11.66,10.44,4.0,-0.09,1.45,4.41,X40,NTT,FMCG
83,VBL,671.64,-20.49,33.0,H-LC,8.50,286271.0,-29571.0,142248.0,0.89,-9.36,49.69,35.67,5.0,-0.21,2.10,3.19,X40N,ATH,FMCG
1,ABB,7934.00,-40.25,46.0,H-LC,7.69,249634.0,-11985.0,131208.0,0.53,-4.58,52.56,45.57,7.0,-0.09,1.83,6.22,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.00,-47.69,40.0,L-SC,40.11,76800.0,-36749.0,76800.0,-1.06,-32.36,100.00,35.27,268.0,-0.48,0.56,87.97,XR,NTT,HOTELS
49,MASFIN,398.61,-20.51,36.0,H-SC,12.30,90255.0,-7725.0,29324.0,-0.59,-7.88,32.49,22.05,152.0,-0.26,0.66,31.19,XR,ATH,FINANCE
61,SAMMAANCAP,326.00,-177.27,70.0,M-SC,6.60,89400.0,-12810.0,106198.0,8.11,-12.53,118.79,91.37,208.0,-0.12,0.66,45.13,XY25,NTT,FINANCE
50,NATIONALUM,244.55,-46.15,52.0,H-MC,11.87,108162.0,7658.0,21449.0,1.74,7.62,19.83,28.96,79.0,0.36,0.79,44.18,MH,ATH,METALS
78,UJJIVANSFB,60.00,90.20,47.0,H-SC,10.25,120231.0,-22248.0,41768.0,0.79,-15.61,34.74,13.70,163.0,-0.53,0.88,41.91,OX40N,NTT,BANKS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.0,1182.30,64.0,M-SC,13.98,174317.0,-6826.0,52295.0,1.78,-3.77,30.00,25.10,235.0,-0.13,1.28,39.70,XY24,NTT,HEALTHCARE
61,SAMMAANCAP,326.0,-177.27,70.0,M-SC,6.60,89400.0,-12810.0,106198.0,8.11,-12.53,118.79,91.37,208.0,-0.12,0.66,45.13,XY25,NTT,FINANCE
81,VAIBHAVGBL,521.0,50.96,47.0,H-SC,5.76,130955.0,-51820.0,162895.0,-0.12,-28.35,124.39,60.77,125.0,-0.32,0.96,17.29,XR,NTT,JEWELLERY
60,SAIL,228.0,45.15,56.0,M-MC,10.74,228992.0,4030.0,163615.0,1.50,1.79,71.45,74.53,192.0,0.02,1.68,34.62,XY24,BTT,STEEL
7,ATULAUTO,844.0,3537.84,47.0,M-SC,7.43,113398.0,-33970.0,85786.0,0.54,-23.05,75.65,35.16,236.0,-0.40,0.83,15.82,XY24,NTT,AUTO


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.27
1,25,43.71
2,50,74.94


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.78
LC    32.56
MC    23.63
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.34
X40      14.80
X40N     11.41
XR       11.10
XY25     10.19
AR        8.30
OX40N     7.87
X200      1.68
MH        1.61
X5K       1.47
SR        1.20
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    29.26
H-LC    26.03
H-MC    20.67
M-SC    13.08
M-LC     5.47
M-MC     2.66
L-SC     1.44
L-LC     1.06
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.05,-6.99,42.30
IT,12.82,-20.20,85.75
FINANCE,10.09,-19.11,71.08
MISC,6.99,-20.65,87.72
BANKS,6.19,-18.02,80.10
PAINTS,5.64,-20.04,37.93
ELECTRICAL,5.49,-9.83,48.36
INSURANCE,3.77,-7.56,47.77
AC,3.56,0.48,16.19


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3239591.0,22
XR,1395666.0,14
AR,1198952.0,9
X40,830783.0,10
X40N,648759.0,9
OX40N,570435.0,10
XY25,448987.0,7
SR,256888.0,2
X5K,116753.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3799935.0,29
M-SC,1461526.0,17
H-MC,1283016.0,15
H-LC,1269679.0,15
M-LC,401957.0,4
M-MC,315199.0,2
L-SC,264368.0,3
L-MC,61579.0,1
L-LC,39786.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1253357.0      6
M-SC       XY24       840179.0      7
H-SC       AR         819571.0      5
           XR         809297.0      7
H-MC       XY24       581789.0      4
H-LC       X40        527894.0      5
H-SC       X40N       320668.0      4
M-MC       XY24       315199.0      2
H-SC       OX40N      267688.0      4
           SR         256888.0      2
H-LC       X40N       240204.0      3
H-MC       X40        225379.0      4
H-LC       AR         208899.0      2
H-MC       XY25       183085.0      2
L-SC       XR         175321.0      2
M-LC       XY24       172619.0      2
M-SC       AR         170482.0      2
           OX40N      140810.0      4
           XR         126347.0      2
H-LC       XY25       119918.0      3
M-LC       X5K        116753.0      1
           XR         112585.0      1
H-MC       XR         110537.0      1
M-SC       XY25       106198.0      1
H-LC       X200        96316.0      1
L-SC       OX40N       89047.0      1
H-MC       X40N        87887.0      2
M-SC       X40         77510.0      1
H-LC       XY24        76448.0      1
H-MC       OX40N       72890.0      1
H-SC       MH          72466.0      1
L-MC       XR          61579.0      1
L-LC       XY25        39786.0      1
H-MC       MH          21449.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 43.0 seconds
